<a href="https://colab.research.google.com/github/Rotem2411/alephBERTgimmelDalet/blob/main/Latent_Dirichlet_Allocation2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import csv
import pandas as pd
import numpy as np
import nltk
import pyLDAvis.gensim_models
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV
from pprint import pprint

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
file_path = 'sentiments.csv'
df = pd.read_csv(file_path)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Take a peek of the data

In [5]:
num_samples = len(df)
print("Total number of samples: ", num_samples)
print(df.head(10))

Total number of samples:  75151
   id                                               text tag category class  \
0   1                    האריות של הצל חזק פה בתגובות...   ש  ECONOMY    b1   
1   2  זמרת תעשייה רק מפרסומות מפורסמת\n  אבל אין קהל...   ש  ECONOMY    b1   
2   4  את לא יורקת לבאר שממנה שתית. יפה יפה, אבל חסרת...   ש  ECONOMY    b1   
3   5  שמעון\n  משתתף בפרסומת למילקי במקום לעורר מודע...   ש  ECONOMY    b1   
4   6  הצחקתם אותי\n  מה כל כך אמייזינג בחברה עם אפס ...   ש  ECONOMY    b1   
5   7  איסו חברת הייטק מצליחה צריכה פרסומות שיבואו לע...   ש  ECONOMY    b1   
6   8           לא מכיר אותה ולא שמעתי אפילו שיר אחד שלה   ש  ECONOMY    b1   
7   9  וואללה לא יודע מה מתלהבים מנגה ארז. זה בכלל לא...   ש  ECONOMY    b1   
8  10  אהובה שגיא\n  נגה גדולה.זמרת מעולה.פרסומת מדלי...   ח  ECONOMY    b1   
9  11  הפרסומת של אמדוקס נראית מאוד קודרת ואפורה, כמו...   ש  ECONOMY    b1   

   total_tags  selected_tag  polarity  
0           2             2       1.0  
1           2     

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# **Data Pre-processing**

We will perform the following steps:


*   **Tokenization**: Split the text into sentences and the sentences into words
*   Remove **punctuation**.
*   All **stopwords** are removed.
*   Words are **lemmatized** — words in third person are changed to first person and verbs in past and future tenses are changed into present.
*   Words are **hebrew** — words with english letters or numbers are removed.

In [6]:
def contains_english_letter(word):
    return any(c.isascii() for c in word)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
def cleanwords(text):
    wn = nltk.WordNetLemmatizer()
    stopword = nltk.corpus.stopwords.words('hebrew')
    tokens = nltk.word_tokenize(text)
    no_stopwords = [word for word in tokens if word not in stopword]
    no_alpha = [word for word in no_stopwords if word.isalpha()]
    no_ascii = [word for word in no_alpha if not contains_english_letter(word)]
    lemm_text = [wn.lemmatize(word) for word in no_ascii]
    return lemm_text

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
text_before_preprocessed  = df['text']
text_after_preprocessed = {}
for i in text_before_preprocessed.index:
  text_after_preprocessed[i] = cleanwords(text_before_preprocessed[i])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
text_comparison = pd.DataFrame({
    'text_before_preprocessed': text_before_preprocessed,
    'text_after_preprocessed': text_after_preprocessed,
})
display(text_comparison.head(5))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,text_before_preprocessed,text_after_preprocessed
0,האריות של הצל חזק פה בתגובות...,"[האריות, הצל, חזק, בתגובות]"
1,זמרת תעשייה רק מפרסומות מפורסמת\n אבל אין קהל...,"[זמרת, תעשייה, מפרסומות, מפורסמת, קהל, אמיתי, ..."
2,"את לא יורקת לבאר שממנה שתית. יפה יפה, אבל חסרת...","[יורקת, לבאר, שממנה, שתית, יפה, יפה, חסרת, עמו..."
3,שמעון\n משתתף בפרסומת למילקי במקום לעורר מודע...,"[שמעון, משתתף, בפרסומת, למילקי, במקום, לעורר, ..."
4,הצחקתם אותי\n מה כל כך אמייזינג בחברה עם אפס ...,"[הצחקתם, אמייזינג, בחברה, אפס, נאמנות, לעובדים]"


In [10]:
text_after_list = []
for i in range(len(text_after_preprocessed)):
    text_after_list.append(' '.join(str(e) for e in text_after_preprocessed[i]))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# **TF-IDF**

In [11]:
idx = []
for i in range(len(text_after_list)):
    idx.append('doc ' + str(i))

# instantiate the vectorizer object
countvectorizer = CountVectorizer()
tfidfvectorizer = TfidfVectorizer()

# convert th documents into a matrix
count_wm = countvectorizer.fit_transform(text_after_list)
tfidf_wm = tfidfvectorizer.fit_transform(text_after_list)

count_tokens = countvectorizer.get_feature_names_out()
tfidf_tokens = tfidfvectorizer.get_feature_names_out()
df_countvect = pd.DataFrame(data = count_wm.toarray(),index = idx, columns = count_tokens)
df_tfidfvect = pd.DataFrame(data = tfidf_wm.toarray(),index = idx, columns = tfidf_tokens)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Create the Document-Word matrix

In [10]:
vectorizer = CountVectorizer(min_df=10, max_features=50000)
data_vectorized = vectorizer.fit_transform(text_after_list)

# **LDA**

In [ ]:
# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=4,               # Number of topics
                                      max_iter=10,                  # Max learning iterations
                                      learning_method='online',
                                      random_state=100,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(df_tfidfvect)
print(lda_model)  # Model attributes

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Diagnose model performance with perplexity and log-likelihood

In [12]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(data_vectorized))
# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(data_vectorized))
# See model parameters
pprint(lda_model.get_params())

Log Likelihood:  -5967938.028268246
Perplexity:  6161.110994333324
{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'online',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 10,
 'mean_change_tol': 0.001,
 'n_components': 4,
 'n_jobs': -1,
 'perp_tol': 0.1,
 'random_state': 100,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


Using GridSearch to determine the best LDA model

In [13]:
# Define Search Param
search_params = {'n_components': [3, 4, 5], 'learning_decay': [.5, .7, .9]}
# Init the Model
lda = LatentDirichletAllocation(max_iter=5, learning_method='online',random_state=0)
# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)
# Do the Grid Search
model.fit(data_vectorized)

KeyboardInterrupt: 

In [ ]:
# Best Model
best_lda_model = model.best_estimator_
# Model Parameters
print("Best Model's Params: ", model.best_params_)
# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)
# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

In [ ]:
import pyLDAvis.gensim_models

vis = pyLDAvis.gensim_models.prepare(best_lda_model, tweets_bow, dictionary=best_lda_model.id2word)
pyLDAvis.save_html(vis, 'new_res/LDA_Visualization_Kimberly_Potter_since_2022-02-15_until_2022-02-20.html')

Dominant topic

In [15]:
# Create Document — Topic Matrix
lda_output = best_lda_model.transform(data_vectorized)
# column names
topicnames = [“Topic” + str(i) for i in range(best_lda_model.n_components)]
# index names
docnames = [“Doc” + str(i) for i in range(len(data))]
# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)
# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic[‘dominant_topic’] = dominant_topic
# Styling
def color_green(val):
 color = ‘green’ if val > .1 else ‘black’
 return ‘color: {col}’.format(col=color)
def make_bold(val):
 weight = 700 if val > .1 else 400
 return ‘font-weight: {weight}’.format(weight=weight)
# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics

SyntaxError: invalid character '“' (U+201C) (<ipython-input-15-038e417f1f42>, line 4)

In [ ]:
# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(best_lda_model.components_)
# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names()
df_topic_keywords.index = topicnames
# View
df_topic_keywords.head()

In [ ]:
# Show top n keywords for each topic
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=30):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords
topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=30)
# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

In [ ]:
Topics = ["topic 1","topic 2","topic 3","topic 4","topic 5",
          "topic 6", "topic 7", "topic 8", "topic 9", "topic 10"]
df_topic_keywords["Topics"]=Topics
df_topic_keywords

In [19]:
lda_model = LatentDirichletAllocation(n_components=4, max_iteration = 1000)
lda_matrix = lda_model.fit_transform(tfidf_wm)

In [25]:
n = 30
for index, topic in enumerate(lda_model.components_):
    print(f'The top {n} words for topic #{index}')
    print([tfidf_tokens[i] for i in topic.argsort()[-n:]])

The top 30 words for topic #0
['מאוד', 'טוב', 'ברוך', 'פשוט', 'הזמן', 'חיפה', 'היו', 'אפשר', 'סוף', 'עכשיו', 'ביבי', 'שלא', 'עוד', 'ולא', 'כסף', 'לעשות', 'מעניין', 'בהצלחה', 'בכל', 'צריך', 'הזה', 'ישראל', 'הכל', 'מילה', 'הרבה', 'כמה', 'אחד', 'הכבוד', 'לך', 'כבר']
The top 30 words for topic #1
['עובד', 'הכי', 'לפיד', 'דבר', 'גדול', 'מחירים', 'נכון', 'לעשות', 'כלום', 'הכל', 'כסף', 'אפשר', 'הזה', 'כתבה', 'ממש', 'כבר', 'שלא', 'ישראל', 'אחד', 'הכבוד', 'מאוד', 'לך', 'כמה', 'המדינה', 'עוד', 'טוב', 'צריך', 'צודק', 'ביבי', 'ולא']
The top 30 words for topic #2
['הזה', 'היו', 'ליברמן', 'הכבוד', 'הכסף', 'כדי', 'בישראל', 'המדינה', 'אחד', 'עכשיו', 'שכר', 'סוף', 'עובדים', 'ממשלה', 'שהוא', 'הרבה', 'הכל', 'רוצה', 'ישראל', 'ממש', 'כבר', 'כמה', 'כסף', 'טוב', 'לך', 'ולא', 'צריך', 'עוד', 'שלא', 'ביבי']
The top 30 words for topic #3
['מאוד', 'היום', 'בדיוק', 'הכל', 'הזה', 'אפשר', 'שום', 'כמה', 'ביבי', 'עבודה', 'דבר', 'פעם', 'בכלל', 'לך', 'המדינה', 'הרבה', 'כבר', 'כסף', 'לקנות', 'זו', 'פשוט', 'טוב', 'הכי', '

In [ ]:
#####write a code to ignore english words
#### sort the list of words

In [14]:
word_array = tfidf_vect.get_feature_names_out()
# Convert the array to a list
word_list = list(word_array)

In [ ]:
text_after_list = []
for value in text_after.values():
    text_after_list.extend(value)
# We indicate that we would like to exclude any words that show up in less than 2 documents and words that are common across 90% of the documents
cv = CountVectorizer(max_df=0.9, min_df=2)
dtm = cv.fit_transform(text_after_list)

lda = LatentDirichletAllocation(n_components=4, random_state=20)
lda.fit(dtm)

n = 30
for index, topic in enumerate(lda.components_):
    print(f'The top {n} words for topic #{index}')
    print([cv.get_feature_names_out()[i] for i in topic.argsort()[-n:]])

The top 30 words for topic #0
['רוצה', 'לכל', 'לעבוד', 'פחות', 'בישראל', 'בכלל', 'שום', 'פעם', 'שהם', 'שנים', 'וגם', 'שיש', 'עכשיו', 'זו', 'דבר', 'הכי', 'ממש', 'הזה', 'הכל', 'אנשים', 'ביבי', 'כסף', 'כמה', 'לך', 'אחד', 'ישראל', 'עוד', 'צריך', 'ולא', 'שלא']
The top 30 words for topic #1
['שכל', 'צריכים', 'צודק', 'הזאת', 'חבל', 'קשה', 'ראש', 'משהו', 'ואז', 'שכר', 'סוף', 'לשלם', 'אחת', 'ברור', 'מבין', 'ממשלה', 'שאתה', 'אדם', 'בדיוק', 'בארץ', 'כתבה', 'עובדים', 'ללא', 'כלום', 'באמת', 'היו', 'לעשות', 'אפשר', 'מאוד', 'טוב']
The top 30 words for topic #2
['הזו', 'גדול', 'אפילו', 'עושים', 'העולם', 'בעולם', 'כנראה', 'אלא', 'מעניין', 'מחירים', 'מס', 'בית', 'הממשלה', 'שזה', 'רוצים', 'הזמן', 'עבודה', 'נראה', 'לקנות', 'במקום', 'נכון', 'הכבוד', 'היום', 'שהוא', 'פשוט', 'כדי', 'בכל', 'המדינה', 'הרבה', 'כבר']


In [ ]:
text_after_list = []
for value in text_after.values():
    text_after_list.extend(value)

# We indicate that we would like to exclude any words that show up in less than 2 documents and words that are common across 90% of the documents
cv = CountVectorizer(max_df=0.9, min_df=2)
dtm = cv.fit_transform(text_after_list)

In [ ]:
lda = LatentDirichletAllocation(n_components=3, random_state=20)
lda.fit(dtm)

LatentDirichletAllocation(n_components=3, random_state=20)

In [ ]:
n = 30
for index, topic in enumerate(lda.components_):
    print(f'The top {n} words for topic #{index}')
    print([cv.get_feature_names_out()[i] for i in topic.argsort()[-n:]])

The top 30 words for topic #0
['שאתה', 'ממשלה', 'הזמן', 'אדם', 'מדינה', 'שזה', 'רוצה', 'תמיד', 'בארץ', 'בדיוק', 'פחות', 'נכון', 'כתבה', 'במקום', 'עובדים', 'שום', 'עכשיו', 'לעשות', 'היו', 'בכל', 'אפשר', 'הכל', 'המדינה', 'טוב', 'כסף', 'כמה', 'לך', 'אחד', 'ישראל', 'צריך']
The top 30 words for topic #1
['מקום', 'המחירים', 'גדול', 'וזה', 'משהו', 'קשה', 'מחיר', 'העולם', 'אפילו', 'מחירים', 'סוף', 'אלא', 'מס', 'בית', 'לכל', 'עבודה', 'באמת', 'ללא', 'כלום', 'שנים', 'דבר', 'הכי', 'ממש', 'מאוד', 'הזה', 'ביבי', 'הרבה', 'כבר', 'עוד', 'שלא']
The top 30 words for topic #2
['הזאת', 'יודע', 'חבל', 'מעניין', 'לשלם', 'כנראה', 'שנה', 'בעולם', 'ברור', 'עושים', 'הממשלה', 'רוצים', 'הכסף', 'בישראל', 'לעבוד', 'נראה', 'בכלל', 'לקנות', 'הכבוד', 'שהם', 'היום', 'וגם', 'פעם', 'שיש', 'שהוא', 'זו', 'פשוט', 'כדי', 'אנשים', 'ולא']


In [ ]:
data = {
    'Topic': [1, 2, 3],
    'Top Words': [
        [cv.get_feature_names_out()[i] for i in lda.components_[0].argsort()[-n:]],
        [cv.get_feature_names_out()[i] for i in lda.components_[1].argsort()[-n:]],
        [cv.get_feature_names_out()[i] for i in lda.components_[2].argsort()[-n:]]
    ]
}
print(pd.DataFrame(data).to_string())

In [ ]:
topic_results = lda.transform(dtm)
topic = pd.DataFrame(topic_results.argmax(axis=1), columns=["Topic"])
df = pd.concat([df['text'], df['tag'], topic], axis=1)
df.head()

KeyError: 'text'